# What should I watch tonight?
## Movie Data ETL Project

#### Team Members: John Clark, Nicole Fejfar, Jason O'Day, Marianne Pagerit


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import requests

In [ ]:
from config import username
from config import password

# Extract
We extracted data from two sources: "The Numbers", a website with movie data, and a CSV of data on streaming services. More details are below.

## Extract Part 1: The Numbers Website Scrape
Source: https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/all-time-inflation-adjusted

This data was originally located on the source website in a table. The first attempt at scraping was unsuccessful and resulted in a 403 error. The raw python code was interpreted by the website as a bot, and the scraping attempt was blocked. We used code to mimic a web browser to allow us to pull the tables.

In [ ]:
# This is the website where we will pull box office numbers for hollywood movies
url = 'https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/all-time-inflation-adjusted'

# These headers will allow us to avoid a 403 error by mimicing a web browser
header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

In [ ]:
# format the request to mimic a web browser
r = requests.get(url, headers=header)

# import tables using pandas
tables = pd.read_html(r.text)

# save the first table as a dataframe
movie_numbers_df = tables[0]

movie_numbers_df.head()

In [ ]:
# This part takes just a few moments to complete
# set the initial count value. This will be appended to the URL for each loop to get the 
# next page of movies.
count = 101

# iterate through the movie pages, updating the counter for the next set of 100 movies
while count < 10001:
    url_string = url + f'/{count}'
#     print(url_string)
    req = requests.get(url_string, headers=header)
    tables = pd.read_html(req.text)
    movie_numbers_df = movie_numbers_df.append(tables[0])
    count = count + 100

# Transform: part 1
The data in this dataframe is already clean and well-formatted. Box Office numbers were formatted as a string due to the inclusion of commas and dollar signs. In order to use these numbers in analysis, we had to convert to the integer type. Our first step was to remove the commas and the dollar sign, and then we were able to convert the string to an integer. Columns were renamed to enable easier importing into our database.

In [ ]:
# Rename columns
movie_numbers_df = movie_numbers_df.rename(columns={'Rank': 'rank', 'Released': 'released', 'Movie': 'movie',
                                            'Total Box Office': 'total_box_office'})

In [ ]:
# verify the number of movies in the dataframe
movie_numbers_df.shape

In [ ]:
# Remove dollar signs to prepare to convert to int
movie_numbers_df['total_box_office'] = movie_numbers_df['total_box_office'].map(lambda x: x.lstrip('$'))
movie_numbers_df['total_box_office'] = movie_numbers_df['total_box_office'].str.replace(',','')

In [ ]:
movie_numbers_df['total_box_office'] = movie_numbers_df['total_box_office'].astype(int)
movie_numbers_df

## Extract Part 2: Streaming Movie Platform CSV
Source: https://www.kaggle.com/ruchi798/movies-on-netflix-prime-video-hulu-and-disney?select=MoviesOnStreamingPlatforms_updated.csv

This data was downloaded from kaggle in CSV format.

In [ ]:
# Load streaming csv file
stream_file = 'MoviesOnStreamingPlatforms_updated.csv'
stream_df = pd.read_csv(stream_file)
stream_df.head(1)

# Transform: Part 2
We dropped the Type and Country columns, as we felt that they would not be useful in our analysis. We then renamed columns for clarity, updating them to be more descriptive (ie "Age" became "suggested_viewing_age".) We also formatted the new column names to more easily map to the database.

We updated the data in each of the streaming service columns (Netflix, Hulu, Prime Video, and Disney Plus) to be more intuitve to users. We replaced '1' with 'Yes', and '0' with 'No'. 

In [ ]:
# Remove extraneous columns
stream_cols = ['ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes',
       'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Directors',
       'Genres', 'Language', 'Runtime']
stream_df = stream_df[stream_cols].copy()

# Rename columns
stream_df = stream_df.rename(columns={'ID': 'id', 'Title': 'title', 'Year': 'released', 'Age': 'suggested_viewing_age',
                                     'IMDb': 'imdb', 'Rotten Tomatoes': 'rotten_tomatoes', 'Netflix': 'netflix',
                                     'Hulu': 'hulu', 'Prime Video': 'prime_video', 'Disney+': 'disney_plus',
                                     'Directors': 'directors', 'Genres': 'genres', 'Language': 'language',
                                     'Runtime': 'runtime'})

In [ ]:
# Change 0's & 1's to more meaningful data
stream_df['netflix'] = stream_df['netflix'].replace(1,'Yes').replace(0,'No')
stream_df['hulu'] = stream_df['hulu'].replace(1,'Yes').replace(0,'No')
stream_df['prime_video'] = stream_df['prime_video'].replace(1,'Yes').replace(0,'No')
stream_df['disney_plus'] = stream_df['disney_plus'].replace(1,'Yes').replace(0,'No')

In [ ]:
stream_df.head()

# Load
We decided to create two tables in our database: 
* streaming, using data from stream_df (originally sourced from the CSV file)
* box_office, using data from movie_numbers_df (originally sourced from the website scrape)

Table joins would use the title and released fields from the streaming table, and the movie and released fields from the box_office table. It's necessary to use both because remakes or movies with similar stories may have the same titles. (Especially in light of all the recent Disney remakes.)

### Create database connection & load DataFrames into database

In [ ]:
connection_string = f"{username}:{password}@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
movie_numbers_df.to_sql(name='box_office', con=engine, if_exists='replace', index=False)

In [ ]:
stream_df.to_sql(name='streaming', con=engine, if_exists='replace', index=False)

In [ ]:
# verify the streaming data loaded correctly into the database
pd.read_sql_query('select * from streaming', con=engine).head()

In [ ]:
# verify the box_office data loaded correctly into the database
pd.read_sql_query('select * from box_office', con=engine).head()